In [1]:
import os
import numpy as np
import pandas as pd
from IPython.display import display
from IPython.core.display import HTML
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import Div
from bokeh.layouts import row

output_notebook()
pd.options.display.float_format = '{:,.5f}'.format
HTML('<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css">')

Loading BokehJS ...

Backtesting of a UNI v2 BTC/ETH LP strategy
---

In [2]:
from terra_algo_backtest.binance_loader import new_binance_client

# Replace these with your Binance API key and secret
client = new_binance_client(os.getenv("BINANCE_API_KEY"), os.getenv("BINANCE_API_SECRET"))

In [3]:
from binance import Client
from terra_algo_backtest.utils import format_df
from terra_algo_backtest.plotting import new_trade_figure

base_pair, quote_pair = 'BTC/BUSD', 'ETH/BUSD'
start, end = '2022-01-01 00:00:00', '2022-12-31 23:59:59'
frequency = Client.KLINE_INTERVAL_1HOUR

df_lunc = client.get_trade_data(base_pair, frequency, start, end)
df_ustc = client.get_trade_data(quote_pair, frequency, start, end)
df_trades = client.create_trade_data(base_pair, quote_pair, 0, frequency, start, end)
show(new_trade_figure(df_lunc, df_ustc, df_trades, "BTC/ETH"))

Function 'plot_price_ratio' executed in 0.0518s
Function 'plot_scatterplot' executed in 0.0136s
Function 'new_trade_figure' executed in 0.0913s


In [4]:
from terra_algo_backtest.market import MarketQuote, new_market
from terra_algo_backtest.simulation import swap_simulation
from terra_algo_backtest.plotting import new_simulation_figure
from terra_algo_backtest.exec_engine import ConstantProductEngine
from terra_algo_backtest.strategy import SimpleUniV2Strategy

liquidity_usd = 10000000
# BTC/BUSD market price
base = MarketQuote("BTC/BUSD", df_trades.price_1.iloc[0])
# ETH/BUSD market price
quote = MarketQuote("ETH/BUSD", df_trades.price_2.iloc[0])
# create a 1,000,000 USD market for BTC/ETH with 0.3% swap fee 
mkt = new_market(liquidity_usd, quote, base, 0.003)
# create a cp swap execution engine
cp_amm = ConstantProductEngine(mkt)
# uni v2 lp strategy with arb to keep the price of the pool in line with mkt
strategy = SimpleUniV2Strategy(cp_amm, arb_enabled=True)
# run simulation
simul = swap_simulation(df_trades, strategy)
# display results
show(new_simulation_figure(mkt, simul))

Function 'trade_summary' executed in 0.0110s
Function 'sim_results' executed in 0.0428s
Function 'swap_simulation' executed in 0.3901s
